# Exploring Early SARS-CoV2 Mutations
Read the chapter 23 in my book.

In section 23.3.3 we deviate from the book because of Jupyter. Instead of a local Jmol installation we use a web-based version.

In section 23.3.4 we are using a different tool, SamTools instead of IGV.

<figure style="width:50%; display: block; margin-left: auto; margin-right: auto;">
  <img src="ace2-spike.png" alt="ace2-spike.png" >
  <figcaption>Mutations in the receptor binding motif have the highest impact on pathogenicity of SARS-CoV-2</figcaption> 
</figure>

---

We are working in the directory *SARS-CoV-2*.

In [1]:
pwd

/home/jupyter-juggler03/SARS


## Download Programs

We need two AWK scripts

In [2]:
cd /home/wuenschi/notebooks/AngBioInfo_2025/AngBioInfo_2025/
mkdir SARS-CoV-2
cd SARS-CoV-2
pwd

bash: cd: /home/wuenschi/notebooks/AngBioInfo_2025/AngBioInfo_2025/: No such file or directory
/home/jupyter-juggler03/SARS/SARS-CoV-2


In [3]:
wget 'https://github.com/awkologist/CompBiol3/raw/main/23_SARS-CoV-2/fasta2tbl'

--2025-09-23 14:47:53--  https://github.com/awkologist/CompBiol3/raw/main/23_SARS-CoV-2/fasta2tbl
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/awkologist/CompBiol3/main/23_SARS-CoV-2/fasta2tbl [following]
--2025-09-23 14:47:53--  https://raw.githubusercontent.com/awkologist/CompBiol3/main/23_SARS-CoV-2/fasta2tbl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [text/plain]
Saving to: ‘fasta2tbl’

fasta2tbl           100%[===================>]     190  --.-KB/s    in 0s      

2025-09-23 14:47:53 (14.2 MB/s) - ‘fasta2tbl’ saved [190/190]



Make the file executable ...

In [4]:
chmod u+x ./fasta2tbl

In [5]:
wget 'https://github.com/awkologist/CompBiol3/raw/main/23_SARS-CoV-2/compare-cov2.awk'

--2025-09-23 14:48:02--  https://github.com/awkologist/CompBiol3/raw/main/23_SARS-CoV-2/compare-cov2.awk
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/awkologist/CompBiol3/main/23_SARS-CoV-2/compare-cov2.awk [following]
--2025-09-23 14:48:02--  https://raw.githubusercontent.com/awkologist/CompBiol3/main/23_SARS-CoV-2/compare-cov2.awk
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2860 (2.8K) [text/plain]
Saving to: ‘compare-cov2.awk’

compare-cov2.awk    100%[===================>]   2.79K  --.-KB/s    in 0s      

2025-09-23 14:48:03 (52.2 MB/s) - ‘compare-cov2.awk’ saved [2860/2860]



## Download Virus Sequences

Download reference genome:

In [6]:
efetch -db nuccore -id NC_045512 -format fasta > wuhan-1.fasta

We create a copy in tab-delimited format:

In [7]:
./fasta2tbl wuhan-1.fasta > wuhan-1.tab

Download from [NCBI](https://www.ncbi.nlm.nih.gov/sars-cov-2/) viruses from Europe, from human hosts, without ambigious characters, complete nucleotide sequences, and a sequence length of exactly 29,903 nt. They are downloaded via the web browser as *sequences.fasta*. Move them into your current working directory.

Detailed instruction are in chapter "**23.2.3 Data and GitHub Repository**"

Move the file (ca 100 MB) from your local computer to JuypterHub. It takes a while – check in the GitHub file browser if the complete file has been uploaded.

Here we print the header for the reference genome:

In [8]:
cut -f 1 wuhan-1.tab
awk -F"\t" '{print $1}' ./wuhan-1.tab

NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome


Rename the file to *cov2-len-29903.fasta*

In [11]:
mv sequences.fasta cov2-len-29903.fasta

In [10]:
pwd

/home/jupyter-juggler03/SARS/SARS-CoV-2


In [12]:
grep -c ">" cov2-len-29903.fasta

3375


## Convert FASTA to TAB and Edit Header

In [13]:
head -1 cov2-len-29903.fasta

>PV612058.1 |2020-04-02|Sweden


In [14]:
./fasta2tbl cov2-len-29903.fasta | sed 's/|/\t/g' > cov2-len-29903.tab

In [15]:
wc -l cov2-len-29903.tab

3375 cov2-len-29903.tab


In [16]:
cut -f 1 cov2-len-29903.tab | head -2

PV612058.1 
PV612059.1 


In [17]:
cut -f 1,3 cov2-len-29903.tab | head -2

PV612058.1 	Sweden
PV612059.1 	Sweden


In [18]:
cut -f 1-3 cov2-len-29903.tab | head -2

PV612058.1 	2020-04-02	Sweden
PV612059.1 	2020-04-03	Sweden


## Analyze Data

In [19]:
cut -f 2 cov2-len-29903.tab | sed 's/:.*//' | sort | uniq -c 
# cut -f 2 cov2-len-29903.tab | sort | uniq -c 

      1 
    142 2020
      6 2020-02
     80 2020-03
      1 2020-03-06
      1 2020-03-09
      2 2020-03-10
      1 2020-03-11
      1 2020-03-13
      1 2020-03-16
      1 2020-03-17
      2 2020-03-18
      3 2020-03-19
      4 2020-03-20
      2 2020-03-21
      1 2020-03-22
      7 2020-03-23
      7 2020-03-24
      1 2020-03-25
      3 2020-03-27
      1 2020-03-28
      5 2020-03-29
      3 2020-03-30
      7 2020-03-31
      4 2020-04-01
      6 2020-04-02
      4 2020-04-03
      3 2020-04-04
     11 2020-04-05
      5 2020-04-06
      9 2020-04-07
     14 2020-04-08
      4 2020-04-09
      1 2020-04-10
      1 2020-04-12
      4 2020-04-13
      5 2020-04-14
     10 2020-04-15
      6 2020-04-16
      9 2020-04-17
     10 2020-04-18
      3 2020-04-19
     12 2020-04-20
      7 2020-04-21
      3 2020-04-22
      4 2020-04-23
      5 2020-04-25
      1 2020-04-26
      1 2020-04-27
      1 2020-04-28
      2 2020-04-29
      5 2020-04-30
      3 2020-05-01
      3 2020-05

In [20]:
egrep "Germany" cov2-len-29903.tab | cut -f 1-3

OK075090.1 	2021-01-08	Germany: Langen
MT358638.1 	2020-02	Germany
MT358639.1 	2020-02	Germany
MT358640.1 	2020-02	Germany
MT358641.1 	2020-02	Germany
MT358642.1 	2020-02	Germany
MT358643.1 	2020-02	Germany


In [21]:
awk -f compare-cov2.awk -v ref=wuhan-1.tab -v seq=cov2-len-29903.tab -v id2=MT358638

# Compared: NC_045512.2 vs MT358638.1  | 12/2019 vs Germany | Wuhan vs 2020-02
MT358638.1  3518: G>T 
MT358638.1  12704: G>T 
MT358638.1  12797: G>A 
MT358638.1  17423: A>G 
MT358638.1  27272: T>C 
MT358638.1  28854: C>T 
# of N's: 0 of 29903
# of exchanges: 6 of 29903


In [22]:
for i in MT358638 MT358639 MT358640 MT358641 MT358642 MT358643; do awk -f compare-cov2.awk -v ref=wuhan-1.tab -v seq=cov2-len-29903.tab -v id2=$i; done

# Compared: NC_045512.2 vs MT358638.1  | 12/2019 vs Germany | Wuhan vs 2020-02
MT358638.1  3518: G>T 
MT358638.1  12704: G>T 
MT358638.1  12797: G>A 
MT358638.1  17423: A>G 
MT358638.1  27272: T>C 
MT358638.1  28854: C>T 
# of N's: 0 of 29903
# of exchanges: 6 of 29903
# Compared: NC_045512.2 vs MT358639.1  | 12/2019 vs Germany | Wuhan vs 2020-02
MT358639.1  241: C>T 
MT358639.1  3037: C>T 
MT358639.1  14408: C>T 
MT358639.1  23403: A>G SPIKE NT:1841 AA:614 -> D
MT358639.1  28881: G>A 
MT358639.1  28882: G>A 
MT358639.1  28883: G>C 
# of N's: 0 of 29903
# of exchanges: 7 of 29903
# Compared: NC_045512.2 vs MT358640.1  | 12/2019 vs Germany | Wuhan vs 2020-02
MT358640.1  241: C>T 
MT358640.1  3037: C>T 
MT358640.1  6228: A>G 
MT358640.1  14408: C>T 
MT358640.1  16289: C>Y 
MT358640.1  23403: A>G SPIKE NT:1841 AA:614 -> D
MT358640.1  28881: G>A 
MT358640.1  28882: G>A 
MT358640.1  28883: G>C 
MT358640.1  28961: C>T 
MT358640.1  29870: C>M 
# of N's: 0 of 29903
# of exchanges: 11 of 29903


In [23]:
awk -f compare-cov2.awk -v ref=wuhan-1.tab -v seq=cov2-len-29903.tab -v id2=OK075090

# Compared: NC_045512.2 vs OK075090.1  | 12/2019 vs Germany: Langen | Wuhan vs 2021-01-08
OK075090.1  241: C>T 
OK075090.1  1059: C>T 
OK075090.1  3037: C>T 
OK075090.1  5143: T>G 
OK075090.1  7843: C>T 
OK075090.1  14408: C>T 
OK075090.1  15920: T>C 
OK075090.1  21846: C>T SPIKE NT:284 AA:95 -> T
OK075090.1  22296: A>G SPIKE NT:734 AA:245 -> H
OK075090.1  23403: A>G SPIKE NT:1841 AA:614 -> D
OK075090.1  23618: A>G SPIKE NT:2056 AA:686 -> S
OK075090.1  26895: C>T 
OK075090.1  27996: G>T 
OK075090.1  28044: G>T 
OK075090.1  29769: C>T 
# of N's: 0 of 29903
# of exchanges: 15 of 29903


In [24]:
# mit head -50 werden statt allen (ca 3.000) nur 50 Sequenzen bearbeitet
for i in $(cut -f 1 cov2-len-29903.tab | head -50 | sed 's/\..*//'); do awk -f compare-cov2.awk -v ref=wuhan-1.tab -v seq=cov2-len-29903.tab -v id2=$i; done > result.txt

In [25]:
egrep -c ">" result.txt

802


In [26]:
egrep -c "SPIKE" result.txt

130


In [27]:
egrep -c "Motif" result.txt

4


In [28]:
egrep "Motif" result.txt | head -5

OX648044.1  22879: C>A SPIKE NT:1317 AA:440 -> N RBDomain RBMotif
OX648046.1  22992: G>A SPIKE NT:1430 AA:477 -> S RBDomain RBMotif
OX648346.1  22992: G>A SPIKE NT:1430 AA:477 -> S RBDomain RBMotif
OX648371.1  22992: G>A SPIKE NT:1430 AA:477 -> S RBDomain RBMotif


In [29]:
egrep "Motif" result.txt | cut -d ' ' -f 7 | sort | uniq -c
echo
egrep "Motif" result.txt | cut -d ' ' -f 7 

      1 AA:440
      3 AA:477

AA:440
AA:477
AA:477
AA:477


Goto an online version of [Jmol](https://lampz.tugraz.at/~hadley/ss1/molecules/moleculeviewer/viewer.php) in your browser of choice. There you can open the Jmol terminal as shown in this video:

<figure style="width:50%; display: block; margin-left: auto; margin-right: auto;">
  <img src="jmol_im_www.gif" alt="jmol_im_www.gif" >
  <figcaption>Using the online version of Jmol.</figcaption>
</figure>

Copy/paste the following code into the Jmol terminal 

```
load =7DF4
spacefill off; wireframe off
cartoon
select :A; color lightgray # ACE2
select :C; color gray # Spike 2
select :D; color darkgray # Spike 3
select :B; color lightblue # Spike 1
select 319-541:B; color blue # RBDomain
select 437-508:B; color red # RBMotif
```

Open the PDB structure 7DF4 with the above script directly in [jmol.php](https://chemapps.stolaf.edu/jmol/jmol.php?pdbid=7df4&script=wireframe&nbsp;off;spacefill&nbsp;off;cartoon;select&nbsp;:A;color&nbsp;lightgray;select&nbsp;:C;color&nbsp;gray;select&nbsp;:D;color&nbsp;darkgray;select&nbsp;:B;color&nbsp;lightblue;select&nbsp;319-541:B;color&nbsp;blue;select&nbsp;437-508:B;color&nbsp;red)

Note to myself: Spaces are replaed by ```&nbsp;``` // No space behind semicolon // No comments

In [30]:
egrep "Motif" result.txt | awk '{print $6}' | sort | uniq -c | sed 's/:/ /' | awk '{if($1<10){print "select "$3":B; spacefill 100; color yellow"}else{print "select "$3":B; spacefill 300; color yellow"}}' | tee -a 7DF4.script

select 440:B; spacefill 100; color yellow
select 477:B; spacefill 100; color yellow


You can now add these Jmol commands to the Jmol terminal and execute them.

## Mapping Genomic Variance

In [34]:
wget https://github.com/lh3/minimap2/releases/latest/download/minimap2

--2025-09-23 15:07:10--  https://github.com/lh3/minimap2/releases/latest/download/minimap2
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/lh3/minimap2/releases/download/v2.30/minimap2 [following]
--2025-09-23 15:07:10--  https://github.com/lh3/minimap2/releases/download/v2.30/minimap2
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 404 Not Found
2025-09-23 15:07:10 ERROR 404: Not Found.



: 8

In [35]:
chmod +x minimap2

chmod: cannot access 'minimap2': No such file or directory


: 1

In [36]:
sudo apt install minimap2

sudo: The "no new privileges" flag is set, which prevents sudo from running as root.
sudo: If sudo is running in a container, you may need to adjust the container configuration to disable the flag.


: 1

In [37]:
minimap2 -x asm5 -a -o sequences.sam wuhan-1.fasta cov2-len-29903.fasta

[M::mm_idx_gen::0.001*1.52] collected minimizers
[M::mm_idx_gen::0.002*1.22] sorted minimizers
[M::main::0.002*1.22] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.002*1.20] mid_occ = 50
[M::mm_idx_stat] kmer size: 19; skip: 19; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.002*1.19] distinct minimizers: 3006 (100.00% are singletons); average occurrences: 1.000; average spacing: 9.948; total length: 29903
[M::worker_pipeline::4.693*2.89] mapped 3375 sequences
[M::main] Version: 2.26-r1175
[M::main] CMD: minimap2 -x asm5 -a -o sequences.sam wuhan-1.fasta cov2-len-29903.fasta
[M::main] Real time: 4.694 sec; CPU: 13.566 sec; Peak RSS: 0.100 GB


Print SAM without sequence:

In [38]:
awk -F"\t" '{ORS=""; for(i=1; i<=NF; i++){if($i!~/[ATCG]{10,}/){print $i" "}}print "\n"}' sequences.sam | head -10

@HD VN:1.6 SO:unsorted GO:query 
@SQ SN:NC_045512.2 LN:29903 
@PG ID:minimap2 PN:minimap2 VN:2.26-r1175 CL:minimap2 -x asm5 -a -o sequences.sam wuhan-1.fasta cov2-len-29903.fasta 
PV612058.1 0 NC_045512.2 1 60 29903M * 0 0 * NM:i:8 ms:i:29743 AS:i:29743 nn:i:0 tp:A:P cm:i:2989 s1:i:29754 s2:i:0 de:f:0.0003 rl:i:0 
PV612059.1 0 NC_045512.2 1 60 29903M * 0 0 * NM:i:10 ms:i:29703 AS:i:29703 nn:i:0 tp:A:P cm:i:2983 s1:i:29723 s2:i:0 de:f:0.0003 rl:i:0 
PV612060.1 0 NC_045512.2 1 60 29903M * 0 0 * NM:i:7 ms:i:29763 AS:i:29763 nn:i:0 tp:A:P cm:i:2992 s1:i:29780 s2:i:0 de:f:0.0002 rl:i:0 
PV612061.1 0 NC_045512.2 1 60 29903M * 0 0 * NM:i:8 ms:i:29743 AS:i:29743 nn:i:0 tp:A:P cm:i:2990 s1:i:29765 s2:i:0 de:f:0.0003 rl:i:0 
OY902096.1 0 NC_045512.2 1 60 29903M * 0 0 * NM:i:21 ms:i:29483 AS:i:29483 nn:i:0 tp:A:P cm:i:2969 s1:i:29635 s2:i:0 de:f:0.0007 rl:i:0 
OX648031.1 0 NC_045512.2 1 60 29903M * 0 0 * NM:i:20 ms:i:29503 AS:i:29503 nn:i:0 tp:A:P cm:i:2961 s1:i:29582 s2:i:0 de:f:0.0007 rl:i:0 
O

The 6th field specifies the changes. "29903M" means that all 29903 nucleotides match (see table 23.1 in my book). Therefore, we print only lines with mutations:

In [39]:
awk -F"\t" '$6!="29903M"{ORS=""; for(i=1; i<=NF; i++){if($i!~/[ATCG]{10,}/){print $i" "}}print "\n"}' sequences.sam | head -10

@HD VN:1.6 SO:unsorted GO:query 
@SQ SN:NC_045512.2 LN:29903 
@PG ID:minimap2 PN:minimap2 VN:2.26-r1175 CL:minimap2 -x asm5 -a -o sequences.sam wuhan-1.fasta cov2-len-29903.fasta 
OX654083.1 0 NC_045512.2 1 60 29894M9S * 0 0 * NM:i:27 ms:i:29354 AS:i:29354 nn:i:0 tp:A:P cm:i:2939 s1:i:29427 s2:i:0 de:f:0.0009 rl:i:0 
OX654115.1 0 NC_045512.2 1 60 29898M5S * 0 0 * NM:i:16 ms:i:29596 AS:i:29596 nn:i:1 tp:A:P cm:i:2973 s1:i:29688 s2:i:0 de:f:0.0005 rl:i:0 
OX654563.1 0 NC_045512.2 1 60 29866M37S * 0 0 * NM:i:23 ms:i:29406 AS:i:29406 nn:i:0 tp:A:P cm:i:2955 s1:i:29542 s2:i:0 de:f:0.0008 rl:i:0 
OX654704.1 0 NC_045512.2 1 60 29889M14S * 0 0 * NM:i:16 ms:i:29569 AS:i:29569 nn:i:0 tp:A:P cm:i:2963 s1:i:29599 s2:i:0 de:f:0.0005 rl:i:0 
OX654810.1 0 NC_045512.2 14 60 13S29890M * 0 0 * NM:i:21 ms:i:29470 AS:i:29470 nn:i:0 tp:A:P cm:i:2958 s1:i:29570 s2:i:0 de:f:0.0007 rl:i:0 
OX655121.1 0 NC_045512.2 1 60 29872M31S * 0 0 * NM:i:19 ms:i:29492 AS:i:29492 nn:i:0 tp:A:P cm:i:2962 s1:i:29602 s2:i:0 d

In [40]:
awk -F"\t" '$6!="29903M"{ORS=""; for(i=1; i<=NF; i++){if($i!~/[ATCG]{10,}/){print $i" "}}print "\n"}' sequences.sam | wc -l

96


And now we print the full lines, including the sequences.

In [49]:
awk -F"\t" '$6!="29903M"{print $0}' sequences.sam > sequences_mutated.sam
# awk -F"\t" '$6!="29903M"{print $1, $6}' sequences.sam > sequences_mutated.sam

In [47]:
ls

7DF4.script	      cov2-len-29903.tab  sequences.sam		 wuhan-1.tab
compare-cov2.awk      fasta2tbl		  sequences_mutated.sam
cov2-len-29903.fasta  result.txt	  wuhan-1.fasta


Convert SAM file to binary format and sort 

In [50]:
samtools view -b sequences_mutated.sam | samtools sort - -o sequences_mutated.sorted.bam

Create index for *sequences.sorted.bam* file for visualization with IGV

In [51]:
samtools index sequences_mutated.sorted.bam

We can now look at the variants with the Samtools viewer ```tview```:

Therefore, change to the **terminal** and run ```samtools tview sequences_mutated.sorted.bam wuhan-1.fasta -p NC_045512.2:22871-23084```

This opens the following screen. The top sequence represents the reference stored in *wuhan-1.fasta*. Each following line represents one virus genome sequence. A dot means that the nucleotide at this position matches the reference. For mutated nucleotides, the nucleotide is shown. You can sroll left and right with the curos keys and can apply commands as shown in the help. ```Q```brings you back to the terminal prompt.

<figure style="width:50%; display: block; margin-left: auto; margin-right: auto;">
  <img src="samtools-tview.png" alt="samtools-tview.png" >
  <figcaption>The terminal based alignment viewer tview of SamTools.</figcaption>
</figure>

## IGV Approach with Xvfb
Xvfb is a virtual frame buffer for the graphical X11-server. That means you can open graphical windows without displaying them. Instead, snapshots can be stored.

In [53]:
bcftools mpileup --max-depth 4000 -a AD -f wuhan-1.fasta -o seq-4000.bcf sequences_mutated.sorted.bam

[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 4000


Now call the significantly variant position with the multiallelic-caller (`-m`) and assume a ploidy of two. Save only variant sites (`-v`) in a VCF formated file (`-O v`).

In [54]:
bcftools call -O v -v -m --ploidy 2 -o seq-4000.vcf seq-4000.bcf

Convert original file with all variant position (without statistical model) to VCF

In [55]:
bcftools view -O v -o seq-4000-all.vcf seq-4000.bcf

Extract all positions that have at least one variant nucleotide.

In [56]:
awk '$5~/[ACGT]/||$0~/^#/{print $0}' seq-4000-all.vcf > seq-4000-snv.vcf

Download GenBank file of reference genome for annotation

In [57]:
efetch -db nuccore -id NC_045512 -format genbank > wuhan-1.gb

Edit GenBank file to be compatible with the FASTA version. 

In [58]:
sed -i 's/NC_045512/NC_045512.2/' wuhan-1.gb

Create GFF3 File *spikeprotein.gff3*:
```
##gff-version 3
##track name="Spike Protein" gffTags=on
NC_045512.2     .       CDS     21563   25383   .       +       .       ID=Spike;Name=SpikeProtein;Color=blue
NC_045512.2     .       CDS     22517   23183   .       +       .       ID=Domain;Name=RBDomain;Color=green
NC_045512.2     .       CDS     22871   23084   .       +       .       ID=Motif;Name=RBMotif;Color=red
```

Call IGV with ```xvfb-run```and the following IGV script file called *igv.bat*:

```
new
snapshotDirectory snapshots
# genome wuhan-1.gb
load spikeprotein.gff3
# load sequences_mutated.sorted.bam
load seq-4000.vcf 
load seq-4000-snv.vcf
snapshot region_full.png
# RBM:
goto NC_045512.2:22871-23084
snapshot region_RBM.png
# RBD:
goto NC_045512.2:22517-23183
snapshot region_RBD.png
exit
```

In [ ]:
# igv -g wuhan-1.gb -l NC_045512.2:22871-23084 sequences.sorted.bam seq-4000.vcf seq-4000-snv.vcf spikeprotein.gff3
xvfb-run -a igv -b igv.bat # notebooks/AngBioInfo_2025/AngBioInfo_2025/SARS-CoV-2

Sep 23, 2025 1:31:17 PM java.util.prefs.FileSystemPreferences$1 run
INFO: Created user preferences directory.
INFO [Sep 23,2025 13:31] [Main] Startup  IGV Version user not_set
INFO [Sep 23,2025 13:31] [Main] Java 21.0.8 (build 21.0.8+9-Ubuntu-0ubuntu124.04.1) 2025-07-15
INFO [Sep 23,2025 13:31] [Main] Java Vendor: Ubuntu https://ubuntu.com/
INFO [Sep 23,2025 13:31] [Main] JVM: OpenJDK 64-Bit Server VM    
INFO [Sep 23,2025 13:31] [Main] OS: Linux 6.8.0-79-generic amd64
INFO [Sep 23,2025 13:31] [Main] IGV Directory: /home/jupyter-juggler03/igv
INFO [Sep 23,2025 13:31] [CommandListener] Listening on port 60151
INFO [Sep 23,2025 13:31] [BatchRunner] Executing batch script: igv.bat
SEVERE [Sep 23,2025 13:31] [UIUtilities] Error invoking runnable
SEVERE [Sep 23,2025 13:31] [UIUtilities] java.lang.reflect.InvocationTargetException
	at java.desktop/java.awt.EventQueue.invokeAndWait(EventQueue.java:1371)
	at java.desktop/java.awt.EventQueue.invokeAndWait(EventQueue.java:1346)
	at java.desktop/

**Attention**: Images do not update automatically:

<figure style="text-align: center;">
  <img src="snapshots/region_full.png" style="width: 30%; display: inline-block; margin-right: 2%;">
  <img src="snapshots/region_RBD.png" style="width: 30%; display: inline-block;">
  <img src="snapshots/region_RBM.png" style="width: 30%; display: inline-block;">
  <figcaption>Window snapshot of IGV with left) the complete virus, middle) the RBD and right) the RBM.</figcaption>
</figure>
<script>
  document.getElementById("myimg").src += '?t=' + new Date().getTime();
</script>

sequences_mutated.sorted.bam